In [1]:
import pandas as pd
import sys
sys.path.append("../..")
import tabularmagic as tm


data = pd.read_csv("data/card.csv")
data["educ_binary"] = data["educ"].apply(lambda x: 1 if x >= 13 else 0)


In [2]:
analyzer = tm.Analyzer(data)

NOTE: No test DataFrame provided. The test DataFrame will be treated as a train 
      DataFrame copy. 
UPDT: Analyzer initialized for dataset 'Unnamed Dataset'. Shapes of train, test 
      DataFrames: (3010, 35), (3010, 35). 


In [3]:
data.head()

,id,nearc2,nearc4,educ,age,fatheduc,motheduc,weight,momdad14,sinmom14,...,wage,enroll,KWW,IQ,married,libcrd14,exper,lwage,expersq,educ_binary
0,2,0,0,7,29,NaN,NaN,158413,1,0,...,548,0,15.0,NaN,1.0,0.0,16,6.306275,256,0
1,3,0,0,12,27,8.0,8.0,380166,1,0,...,481,0,35.0,93.0,1.0,1.0,9,6.175867,81,0
2,4,0,0,12,34,14.0,12.0,367470,1,0,...,721,0,42.0,103.0,1.0,1.0,16,6.580639,256,0
3,5,1,1,11,27,11.0,12.0,380166,1,0,...,250,0,25.0,88.0,1.0,1.0,10,5.521461,100,0
4,6,1,1,12,34,8.0,7.0,367470,1,0,...,729,0,34.0,108.0,1.0,0.0,16,6.591674,256,0


In [4]:
causal_model = analyzer.causal_model(
    treatment="educ_binary",
    outcome="lwage",
    confounders=[
        "exper", "expersq", "black", "smsa", "south", "smsa66", "reg662", "reg663",
        "reg664", "reg665", "reg666", "reg667", "reg668", "reg669"
    ]
)

In [3]:
analyzer.regress(
    models=[
        tm.ml.LinearR('l2'),
        tm.ml.TreesR("adaboost")
    ],
    target='lwage',
)

UPDT: Evaluating model 'LinearR(l2)'. 
WARN: Train data: dropped 1410 rows with missing values out of a total of 3010 rows. 
PROG: Fitting 'LinearR(l2)'. Search method: OptunaSearchCV (100 trials, 500 total fits). 
       
WARN: Test data: dropped 1410 rows with missing values out of a total of 3010 rows. 
UPDT: Successfully evaluated model 'LinearR(l2)'. 
UPDT: Evaluating model 'TreesR(adaboost)'. 
WARN: Train data: dropped 1410 rows with missing values out of a total of 3010 rows. 
PROG: Fitting 'TreesR(adaboost)'. Search method: OptunaSearchCV (100 trials, 500 total 
      fits).  
WARN: Test data: dropped 1410 rows with missing values out of a total of 3010 rows. 
UPDT: Successfully evaluated model 'TreesR(adaboost)'. 


ML Regression Report
----------------------------------------------------------------------------------------
Target variable:
  'lwage' 
                                                                                        
Predictor variables:
  'IQ', 'KWW', 'age', 'black', 'educ', 'educ_binary', 'enroll', 'exper', 'expersq', 
  'fatheduc', 'id', 'libcrd14', 'married', 'momdad14', 'motheduc', 'nearc2', 'nearc4', 
  'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668', 
  'reg669', 'sinmom14', 'smsa', 'smsa66', 'south', 'south66', 'step14', 'wage', 'weight' 
                                                                                        
Models evaluated:
  'LinearR(l2)', 'TreesR(adaboost)' 
                                                                                        
Feature selectors:
  None 
----------------------------------------------------------------------------------------
Best models:
  1. 'TreesR(adaboost)'                     

In [5]:
causal_model.estimate_ate(method="weighted_regression")

CausalReport(estimand='ate', estimate=0.228643, estimate_se=0.022021, treatment_var='educ_binary', outcome_var='lwage', confounders=['exper', 'expersq', 'black', 'smsa', 'south', 'smsa66', 'reg662', 'reg663', 'reg664', 'reg665', 'reg666', 'reg667', 'reg668', 'reg669'])

In [8]:
causal_model.estimate_ate(method="outcome_regression")

Causal Effect Estimation Report
----------------------------------------------------------------------------------------
Estimated Avg Tmt Effect: 0.2385            Std Err: 0.0173
----------------------------------------------------------------------------------------
Treatment variable:
  'educ_binary'
                                                                                        
Outcome variable:
  'lwage'
                                                                                        
Confounders:
  'exper', 'expersq', 'black', 'smsa', 'south', 'smsa66', 'reg662', 'reg663', 'reg664', 
  'reg665', 'reg666', 'reg667', 'reg668', 'reg669' 